In [1]:
import torch
from playgenie.model import VAE
from playgenie.utils.qdrant_search import QdrantSearch

C:\Users\BracklyMurunga\Desktop\AI SUNDAYS\PlayGenie\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\BracklyMurunga\Desktop\AI SUNDAYS\PlayGenie\src\playgenie\utils\qdrant_search.py:31: UserWarning: Local mode is not recommended for collections with more than 20,000 points. Current collection contains 37963 points. Consider using Qdrant in Docker or Qdrant Cloud for better performance with large datasets.
  self.client.upload_collection(


In [2]:
model  = VAE(input_size=384, hidden_size=10,latent_size=2,encoder_n_heads=2,decoder_n_la=2)
optim = torch.optim.Adam(model.parameters())
search_client = QdrantSearch()

In [3]:
sum([p.numel() for p in model.parameters()])

747568

In [4]:
preds,mu,log_var=model(torch.zeros(1,6,384))
preds.shape

torch.Size([1, 6, 384])

In [3]:
torch.cat([torch.zeros(384).view(1,-1),torch.zeros(384).view(1,-1)],dim=0).shape

torch.Size([2, 384])

In [7]:
torch.cat([song.view(1,-1) for song in [torch.zeros(384)for _ in range(1000)]],dim=1).shape

torch.Size([1, 384000])

In [4]:
playlist=[torch.zeros(384)for _ in range(1)]

for _ in range(10):
    song = model.generate(playlist=playlist)
    song_list = search_client.get_item(song_vector=song)
    print('1.',song_list[0].payload['song'],'|',song_list[0].score)
    playlist.append(torch.tensor(song))

1. las cuatro ranas by el viejo paulino y su gente | 0.1561902704066791
1. las cuatro ranas by el viejo paulino y su gente | 0.1497566500289921
1. las cuatro ranas by el viejo paulino y su gente | 0.14609889535609394
1. she's a mess by the real kids | 0.13469537521049119
1. las cuatro ranas by el viejo paulino y su gente | 0.15125819192267093
1. kitty - demo by the presidents of the united states of america | 0.1633420690734208
1. las cuatro ranas by el viejo paulino y su gente | 0.14409032448423348
1. las cuatro ranas by el viejo paulino y su gente | 0.14700369590475248
1. las cuatro ranas by el viejo paulino y su gente | 0.14812152030752215
1. kitty - demo by the presidents of the united states of america | 0.14913741817375406
